## Data Analysis: eBay Kleinanzeigen 

A dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and [uploaded to Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). The course creators made a few modifications from the original dataset:

* Sampled 50,000 data points from the full dataset. 
* Dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with).

### Description of the data set

The columns found in the dataset.

|Columns| Description|
|---|---|
|dateCrawled| When this ad was first crawled. All field-values are taken from this date.|
|name| Name of the car.|
|seller | Whether the seller is private or a dealer.|
|offerType| The type of listing|
| price| The price on the ad to sell the car|
| abtest | Whether the listing is included in an A/B test|
|vehicleType| The vehicle Type.|
|yearOfRegistration | The year in which the car was first registered.|
| gearbox | The transmission type|
|powerPS | The power of the car in PS|
| model | The car model name|
|kilometer | How many kilometers the car has driven|
|monthOfRegistration | The month in which the car was first registered.|
| fuelType | What type of fuel the car uses|
|brand |The brand of the car.|
|notRepairedDamage | If the car has a damage which is not yet repaired.|
|dateCreated | The date on which the eBay listing was created.|
|nrOfPictures | The number of pictures in the ad.|
| postalCode | The postal code for the location of the vehicle.|
|lastSeenOnline | When the crawler saw this ad last online.|

### Aim
The aim of this project is to clean the data and analyze the included used car listings. 

## Preparation

In [1]:
# import libraries
import pandas as pd
import numpy as np

#read the file
autos = pd.read_csv("autos.csv", encoding ="Latin-1")

In [2]:
#inspect first 3 rows
autos.head(3)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


In [3]:
#get basic info about the dataframe
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
#check values in potential bool columns
autos["notRepairedDamage"].unique()

array(['nein', nan, 'ja'], dtype=object)

In [5]:
autos["abtest"].unique()

array(['control', 'test'], dtype=object)

Observations
* There are 20 columns in the dataset most are `object` type. 
* Several of those could be turned into integers, or datetime objects: `dateCrawled`, `price`, `dateCreated`, `lastSeen`
* `notRepairedDamage` and `abtest` columns could also potentially be turned into boolean values since they require yes/no answers.
* Four columns contain some null values: `fuelType`, `notRepairedDamage`, `vehicleType`, `model`. These values might be unknown. 

We might additionally want to split:
* Date columns into date & time columns
* Durn the `odometer` column into integers.

### Converting names
The column names use camelcase instead of Python's preferred snakecase.

We'll convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [6]:
index = autos.columns
print(index)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [7]:
import re

#create a funciton to convert names into lowercase
def convert(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

#new list of names with lowercase
new_list = []
for word in index:
    convert(word)
    new_list.append(word)
print(new_list)

['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'odometer', 'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen']


In [8]:
#rename columns
autos.rename(columns={'dateCrawled': "date_crawled", 'offerType':'offer_type','vehicleType':"vehicle_type", 'yearOfRegistration':'registration_year', 'powerPS':"power_PS", 'monthOfRegistration':'registration_month', "fuelType":'fuel_type','notRepairedDamage':"unrepaired_damage", 'dateCreated':'ad_created', 'nrOfPictures':'nr_of_pictures', 'postalCode':'postal_code', 'lastSeen':'last_seen'},inplace = True)

In [9]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [10]:
#hck if worked
autos.head(2)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08


## Exploration for cleaning prep
Let's determine what other cleaning tasks need to be done. Initially we will look for: 
- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis. 
- Examples of numeric data stored as text which can be cleaned and converted.

In [11]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-05 16:57:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Based on the above we can see:
* The column `nr_of_pictures` has only null values and can be removed. 
* As suggested above there are columns which only have two unique values, which means they can potentially be converted to booleans.
* `registration_year`, `registration_month`, `postal_code`, and `power_PS` are int64 columns yet return NaN values in some stats, this needs to be checked. 
* `ad_created` min values is 1946 whcih seems implausible.

In [12]:
#chech for NaNs in registration year & month column
autos["registration_year"].isnull().sum()

0

In [13]:
autos["registration_month"].isnull().sum()

0

In [14]:
autos["postal_code"].isnull().sum()

0

In [15]:
autos["power_PS"].isnull().sum()

0

As noted before, there are two columns with numerical values stored as text: price in `price` and km in `odometer`. We'll clean these columns and rename them. 

In [16]:
#convert price into a numeric column
autos["price"] = (autos["price"]
                 .str.replace("$","")
                 .str.replace(",","")                  
                 .str.strip()
                 .astype(int)
                 )

In [17]:
#convert odometer into a numeric col
autos["odometer"] = (autos["odometer"]
                    .str.replace("km", "")
                    .str.replace(",","")
                    .str.strip()
                    .astype(int)
                    )
autos.rename(columns= {"odometer":"odometer_km"}, inplace=True)

In [18]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,5.000000e+04,50000,44905,50000.000000,47320,50000.000000,47242,50000.000000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-05 16:57:05,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,NaN,25756,12859,NaN,36993,NaN,4024,NaN,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,9.840044e+03,NaN,NaN,2005.073280,NaN,116.355920,NaN,125732.700000,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,4.811044e+05,NaN,NaN,105.712813,NaN,209.216627,NaN,40042.211706,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1.100000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


## Removing outliers
We'll explore the data, specifically looking for data that doesn't look right, in order to remove potential outliers.

We'll start by analyzing the `odometer_km` and `price` columns.

### Price column

In [19]:
autos["price"].unique().shape

(2357,)

In [20]:
autos["price"].describe().apply(lambda x: '{:.0f}'.format(x))

count       50000
mean         9840
std        481104
min             0
25%          1100
50%          2950
75%          7200
max      99999999
Name: price, dtype: object

#### IQR 

To calculate the outliers we calculate the interquartile range. Subrtracting the value of the lower quartile (2950) from the upper quartile (7200). 

To find minor outliers we identify the inner fences of the data, multipl the interquartlie range by 1.5. 

In [21]:
Q1 = autos["price"].quantile(0.25)
Q3 = autos["price"].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

6100.0


In [22]:
lower = Q1 - 1.5 * IQR
higher = Q3 + 1.5 * IQR

print(lower, higher)

-8050.0 16350.0


This suggests for the price column we should drop all data points lower than -7775 and higher than 16350.

In addition to this consideraiton common sense dictates that price values can't be negative or 0. Given the ebay is an auction site opening bids cound likely be form 1 up. So we'll adjust the bounds and keep all values between 1 and 16350.

#### Use Z score

*Haven't used this methods but keeping the section for reference*

We could also use the Z score to remove the outliers. Z scores range from -3 to 3 and tell us how far a particular score is away from the mean. Eg. A Z score of -2 is 2 standrd deviations away from the mean, to the left of the distribution curve. A Z score of 2 is 2 standrd deviations away from the mean, to the right of the distribution curve. 

In [23]:
from scipy import stats
z = np.abs(stats.zscore(autos["price"]))
z

array([0.01006038, 0.00278538, 0.00176688, ..., 0.00698391, 0.02714606,
       0.01785502])

In [24]:
z1 = z[z>2]
z1

array([  2.05811649,  23.0747888 ,   2.68169011,  20.7652648 ,
         2.54568084,  23.0747888 ,  25.64092496,   2.05809778,
        25.64092496, 207.8367251 ,  56.77074704,   2.05811649,
        25.64092496,   8.06519108])

#### Dropping the values

In [25]:
#drop rows where prices don't fall in the indicated range
autos = autos[autos["price"].between(1,16350)]

In [26]:
autos["price"].unique().shape

(1532,)

In [27]:
autos["price"].describe()

count    44795.000000
mean      4089.433620
std       3841.429247
min          1.000000
25%       1150.000000
50%       2700.000000
75%       5999.000000
max      16350.000000
Name: price, dtype: float64

## Odometer column

In [28]:
autos["odometer_km"].describe()

count     44795.000000
mean     129762.361871
std       36398.554742
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [29]:
autos["odometer_km"].value_counts()

150000    30685
125000     4722
100000     1864
90000      1503
80000      1178
70000       954
60000       874
5000        733
50000       704
40000       527
30000       487
20000       438
10000       126
Name: odometer_km, dtype: int64

There don't seem to be any outliers here. 

## Date column
We'll now explore the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

The `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values. 

We need to convert the data into a numerical representation so we can understand it quantitatively.

These columns all represent full timestamp values, like so:

In [30]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [31]:
#select 10 first chars in a column
date_crawled = autos['date_crawled'].str[:10]
date_crawled[0:5]

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: date_crawled, dtype: object

In [32]:
#display % of values for each date
#sort the values in ascending order
date_crawled.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025516
2016-03-06    0.013997
2016-03-07    0.036098
2016-03-08    0.033642
2016-03-09    0.033129
2016-03-10    0.032705
2016-03-11    0.032481
2016-03-12    0.037393
2016-03-13    0.015604
2016-03-14    0.036991
2016-03-15    0.034178
2016-03-16    0.029959
2016-03-17    0.031990
2016-03-18    0.012769
2016-03-19    0.034200
2016-03-20    0.037705
2016-03-21    0.037192
2016-03-22    0.032727
2016-03-23    0.032481
2016-03-24    0.029378
2016-03-25    0.032013
2016-03-26    0.032593
2016-03-27    0.030718
2016-03-28    0.034781
2016-03-29    0.034200
2016-03-30    0.033865
2016-03-31    0.031588
2016-04-01    0.032794
2016-04-02    0.034982
2016-04-03    0.038531
2016-04-04    0.036366
2016-04-05    0.012948
2016-04-06    0.003170
2016-04-07    0.001317
Name: date_crawled, dtype: float64

The site was crawled between March 5 and April 7 2016. Most days had ~3% of the site crawled. 

In [33]:
#ad created column
#select 10 first chars in a column
ad_created = autos.loc[:,'ad_created'].str[:10]
ad_created[0:5]

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: ad_created, dtype: object

In [34]:
#display % of values for each date
#sort the dates in ascending order
ad_created.value_counts(normalize=True, dropna=False).sort_index()

2015-08-10    0.000022
2015-09-09    0.000022
2015-11-10    0.000022
2015-12-05    0.000022
2015-12-30    0.000022
                ...   
2016-04-03    0.038799
2016-04-04    0.036701
2016-04-05    0.011653
2016-04-06    0.003237
2016-04-07    0.001183
Name: ad_created, Length: 73, dtype: float64

In [35]:
ad_created.describe()

count          44795
unique            73
top       2016-04-03
freq            1738
Name: ad_created, dtype: object

In [36]:
#display % of values for each date
#sort the values in descending order
ad_created.value_counts(normalize=True, dropna=False, ascending=False).head(50)

2016-04-03    0.038799
2016-03-20    0.037772
2016-03-21    0.037370
2016-03-12    0.037236
2016-04-04    0.036701
2016-03-14    0.035674
2016-03-28    0.035049
2016-03-07    0.034803
2016-04-02    0.034691
2016-03-29    0.034044
2016-03-15    0.033999
2016-03-30    0.033664
2016-03-08    0.033575
2016-03-09    0.033196
2016-03-19    0.033084
2016-04-01    0.032838
2016-03-11    0.032838
2016-03-26    0.032615
2016-03-22    0.032571
2016-03-10    0.032414
2016-03-23    0.032347
2016-03-25    0.032102
2016-03-31    0.031633
2016-03-17    0.031521
2016-03-27    0.030561
2016-03-16    0.030450
2016-03-24    0.029356
2016-03-05    0.023128
2016-03-13    0.016855
2016-03-06    0.015337
2016-03-18    0.013528
2016-04-05    0.011653
2016-04-06    0.003237
2016-03-04    0.001496
2016-04-07    0.001183
2016-03-03    0.000871
2016-02-28    0.000201
2016-02-29    0.000156
2016-02-27    0.000134
2016-03-01    0.000112
2016-02-23    0.000089
2016-03-02    0.000089
2016-02-25    0.000067
2016-02-21 

The ads were created between 2015-08-10 and 2016-03-10. Most ads were places from mid March 2016 to early April 2016.

In [37]:
#last seen column
#select 10 first chars in a column
last_seen = autos.loc[:,'last_seen'].str[:10]

In [38]:
#display % of values for each date
#sort the dates in ascending order
last_seen.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001161
2016-03-06    0.004576
2016-03-07    0.005693
2016-03-08    0.007903
2016-03-09    0.010068
2016-03-10    0.011162
2016-03-11    0.013037
2016-03-12    0.024936
2016-03-13    0.009376
2016-03-14    0.012859
2016-03-15    0.016274
2016-03-16    0.016966
2016-03-17    0.029043
2016-03-18    0.007568
2016-03-19    0.016475
2016-03-20    0.021230
2016-03-21    0.021185
2016-03-22    0.021922
2016-03-23    0.019109
2016-03-24    0.020449
2016-03-25    0.020002
2016-03-26    0.017234
2016-03-27    0.016207
2016-03-28    0.021699
2016-03-29    0.023016
2016-03-30    0.025472
2016-03-31    0.024333
2016-04-01    0.023306
2016-04-02    0.025226
2016-04-03    0.025673
2016-04-04    0.025047
2016-04-05    0.121286
2016-04-06    0.214332
2016-04-07    0.126175
Name: last_seen, dtype: float64

In [39]:
last_seen.value_counts().head(20)

2016-04-06    9601
2016-04-07    5652
2016-04-05    5433
2016-03-17    1301
2016-04-03    1150
2016-03-30    1141
2016-04-02    1130
2016-04-04    1122
2016-03-12    1117
2016-03-31    1090
2016-04-01    1044
2016-03-29    1031
2016-03-22     982
2016-03-28     972
2016-03-20     951
2016-03-21     949
2016-03-24     916
2016-03-25     896
2016-03-23     856
2016-03-26     772
Name: last_seen, dtype: int64

Users were seen last betweeen 2016-03-05 and 2016-04-07. Days with higher than usual traffic were between April 5 and April 7, inclusive. 

In [40]:
#registration_year

autos['registration_year'].describe()

count    44795.00000
mean      2003.97339
std         74.81955
min       1000.00000
25%       1999.00000
50%       2003.00000
75%       2007.00000
max       9999.00000
Name: registration_year, dtype: float64

There seem to be wrong values in the `registration_year` column, the min and max values (year 1000 and 9999) are unrealistic. 

A car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [41]:
#removing values outside of the range 1990-2016
autos = autos[autos["registration_year"].between(1990, 2016, inclusive=True)]

In [42]:
autos["registration_year"].describe()

count    41822.000000
mean      2002.997083
std          5.503330
min       1990.000000
25%       1999.000000
50%       2003.000000
75%       2007.000000
max       2016.000000
Name: registration_year, dtype: float64

In [43]:
#calculate the distribution of the remaining values 
autos["registration_year"].value_counts(normalize = True, ascending=False)

2000    0.075128
1999    0.069055
2005    0.069055
2003    0.063818
2004    0.063603
2001    0.062551
2006    0.061762
2002    0.058821
1998    0.056191
2007    0.050858
2008    0.047463
1997    0.046483
2009    0.043231
1996    0.032567
2010    0.029315
1995    0.029171
2016    0.028167
2011    0.026398
2012    0.019129
1994    0.014825
1993    0.009947
2013    0.009469
1992    0.008584
1990    0.008130
1991    0.007891
2014    0.005763
2015    0.002630
Name: registration_year, dtype: float64

Th etop values for registration year are:
2000, 1999, 2005, 2003, 2004, 2001, 2006. Each covers >6% of all values in the dataset.

## Exploring brands

Aggregate the columns based on brands. 

In [44]:
#list all brands and number of cars in each
autos["brand"].value_counts()

volkswagen        8955
opel              4868
bmw               4363
mercedes_benz     3567
audi              3285
ford              3084
renault           2166
peugeot           1381
fiat              1160
seat               819
skoda              725
mazda              686
nissan             669
smart              660
citroen            632
toyota             564
hyundai            453
volvo              382
mitsubishi         372
honda              348
mini               329
kia                299
alfa_romeo         288
suzuki             272
sonstige_autos     258
chevrolet          221
chrysler           156
dacia              123
daihatsu           116
subaru              91
jeep                82
saab                72
daewoo              70
rover               61
porsche             58
land_rover          54
jaguar              52
lancia              46
lada                22
trabant             13
Name: brand, dtype: int64

In [45]:
#check basic stats
autos['brand'].describe()

count          41822
unique            40
top       volkswagen
freq            8955
Name: brand, dtype: object

In [46]:
#display how much % of the dataset belongs to each brand 
#assign to a new series
#sort in ascending order & show the top 30
brand_series=autos['brand'].value_counts(normalize=True)
brand_series.head(30)

volkswagen        0.214122
opel              0.116398
bmw               0.104323
mercedes_benz     0.085290
audi              0.078547
ford              0.073741
renault           0.051791
peugeot           0.033021
fiat              0.027737
seat              0.019583
skoda             0.017335
mazda             0.016403
nissan            0.015996
smart             0.015781
citroen           0.015112
toyota            0.013486
hyundai           0.010832
volvo             0.009134
mitsubishi        0.008895
honda             0.008321
mini              0.007867
kia               0.007149
alfa_romeo        0.006886
suzuki            0.006504
sonstige_autos    0.006169
chevrolet         0.005284
chrysler          0.003730
dacia             0.002941
daihatsu          0.002774
subaru            0.002176
Name: brand, dtype: float64

### Removing
For the later part of the analysis we will remove the brands that take less than 2% of the distribution. We'll pick only those that take over 2%.

In [47]:
#pick values that are over 2% and drop NaNs in the process
brands_over_2 = brand_series.where(brand_series >= 0.02).dropna()

#set the index of the df as an array to have a list of brands over 2%
brands_over_2 = brands_over_2.index
brands_over_2

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat'],
      dtype='object')

In [48]:
#create a dictionary of brand + mean price
top_brands = {}

for brand in brands_over_2: 
    #find the brand in the autos dataframe
    brand_auto = autos[autos["brand"] == brand]
    #calc the mean
    mean_price = brand_auto["price"].mean()
    top_brands[brand] = int(mean_price)
    
top_brands

{'volkswagen': 4186,
 'opel': 2697,
 'bmw': 5651,
 'mercedes_benz': 5239,
 'audi': 5759,
 'ford': 2915,
 'renault': 2267,
 'peugeot': 2962,
 'fiat': 2644}

The most expensive brands are audi, bmw, and mercedes-benz, mid-pric is volksvagen. After a big gap the prices fall in the 2000s 5th an 6th position taken by peugeot and ford. Then opel, fiat and renault.

In [49]:
#turn the results into a df
top_brands_df=pd.DataFrame(top_brands.items(), columns=['brand', 'mean_price'])
top_brands_df

,brand,mean_price
0,volkswagen,4186
1,opel,2697
2,bmw,5651
3,mercedes_benz,5239
4,audi,5759
5,ford,2915
6,renault,2267
7,peugeot,2962
8,fiat,2644


## Mean mileage & mean price 
Let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price.

### Calculate mean mileage for top brands

In [50]:
autos['odometer_km'].describe()

count     41822.000000
mean     130232.054899
std       35778.990002
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [51]:
#how much mileage belongs to each brand in %
brand_km=autos['odometer_km'].value_counts(normalize=True)

In [52]:
#how much mileage belongs to each brand
mean_mileage = {}

for brand in brands_over_2: 
    #pick the brands from the top brands
    brand_auto = autos[autos["brand"] == brand]
    #calc the mean
    mean_miles = brand_auto["odometer_km"].mean()
    mean_mileage[brand] = int(mean_miles)

#turn a dictionary to a DF
mean_mil_df= pd.DataFrame(mean_mileage.items(), columns=['brand', 'mean_mileage'])
mean_mil_df

,brand,mean_mileage
0,volkswagen,133300
1,opel,131036
2,bmw,139049
3,mercedes_benz,138728
4,audi,139636
5,ford,127534
6,renault,129141
7,peugeot,127918
8,fiat,118625


In [53]:
#merge the dfs into one
cols_to_use = mean_mil_df.columns.difference(top_brands_df.columns)
dfNew = pd.merge(top_brands_df, mean_mil_df[cols_to_use], left_index=True, right_index=True, how='outer')
dfNew

,brand,mean_price,mean_mileage
0,volkswagen,4186,133300
1,opel,2697,131036
2,bmw,5651,139049
3,mercedes_benz,5239,138728
4,audi,5759,139636
5,ford,2915,127534
6,renault,2267,129141
7,peugeot,2962,127918
8,fiat,2644,118625


## Data cleaning, next steps

Split all cloumns that have dates into date and hour. Then convert all dates into numerical columns.

In [55]:
autos["ad_created"] = (autos["ad_created"]
                         .str.split(n=1, expand= True)
                         .iloc[:,0]
                         .str.replace("-","")
                        )   

In [56]:
autos['date_crawled_day'], autos['date_crawled_time'] = autos['date_crawled'].str.split(' ', 1).str

autos = autos.drop(["date_crawled"], axis =1)

In [57]:
autos["date_crawled_day"] = (autos["date_crawled_day"]
                         .str.replace("-","")                         
                         .str.split(n=1, expand= True)
                         .iloc[:,0]
                         .astype(int)
                        )   
autos["date_crawled_time"] = (autos["date_crawled_time"]
                         .str.replace(":","")                         
                         .str.split(n=1, expand= True)
                         .iloc[:,0]
                         .astype(int)
                        ) 

In [58]:
#same process with the "last seen" column
autos['last_seen_day'], autos['last_seen_time'] = autos['last_seen'].str.split(' ', 1).str

autos = autos.drop(["last_seen"], axis =1)

In [59]:
autos["last_seen_day"] = (autos["last_seen_day"]
                         .str.replace("-","")                         
                         .str.split(n=1, expand= True)
                         .iloc[:,0]
                         .astype(int)
                        )   
autos["last_seen_time"] = (autos["last_seen_time"]
                         .str.replace(":","")                         
                         .str.split(n=1, expand= True)
                         .iloc[:,0]
                         .astype(int)  
                        ) 

autos.head()

,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,...,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,date_crawled_day,date_crawled_time,last_seen_day,last_seen_time
0,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,...,lpg,peugeot,nein,20160326,0,79588,20160326,174746,20160406,64554
1,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,...,benzin,bmw,nein,20160404,0,71034,20160404,133856,20160406,144508
2,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,...,benzin,volkswagen,nein,20160326,0,35394,20160326,185724,20160406,201537
3,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,...,benzin,smart,nein,20160312,0,33729,20160312,165810,20160315,31628
4,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,...,benzin,ford,nein,20160401,0,39218,20160401,143850,20160401,143850


In [67]:
#double-check the type of the new columns
autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41822 entries, 0 to 49999
Data columns (total 21 columns):
brand                 41822 non-null object
model                 41667 non-null object
seller                41822 non-null object
offer_type            41822 non-null object
price                 41822 non-null int64
abtest                41822 non-null object
vehicle_type          39265 non-null object
registration_year     41822 non-null int64
gearbox               39915 non-null object
power_PS              41822 non-null int64
odometer_km           41822 non-null int64
registration_month    41822 non-null int64
fuel_type             38747 non-null object
unrepaired_damage     34059 non-null object
ad_created            41822 non-null object
nr_of_pictures        41822 non-null int64
postal_code           41822 non-null int64
date_crawled_day      41822 non-null int64
date_crawled_time     41822 non-null int64
last_seen_day         41822 non-null int64
last_seen_time       

### Extracting models
From the name column we can extract the model, or rather eliminate the brand, which already has its own column.

In [61]:
#see how the names are displayed
autos["name"].head(5)

0                     Peugeot_807_160_NAVTECH_ON_BOARD
1           BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik
2                           Volkswagen_Golf_1.6_United
3    Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...
4    Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...
Name: name, dtype: object

In [62]:
#create a column "model" then drop rdundant columns
autos['to_drop'], autos['model'] = autos['name'].str.split('_', 1).str
autos = autos.drop(['to_drop'], axis =1)
autos = autos.drop(['name'], axis =1)

autos["model"] = (autos["model"]
                 .str.replace("_"," ")
                 .str.strip()
                 )
autos.head()

,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,...,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,date_crawled_day,date_crawled_time,last_seen_day,last_seen_time
0,privat,Angebot,5000,control,bus,2004,manuell,158,807 160 NAVTECH ON BOARD,150000,...,lpg,peugeot,nein,20160326,0,79588,20160326,174746,20160406,64554
1,privat,Angebot,8500,control,limousine,1997,automatik,286,740i 4 4 Liter HAMANN UMBAU Mega Optik,150000,...,benzin,bmw,nein,20160404,0,71034,20160404,133856,20160406,144508
2,privat,Angebot,8990,test,limousine,2009,manuell,102,Golf 1.6 United,70000,...,benzin,volkswagen,nein,20160326,0,35394,20160326,185724,20160406,201537
3,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,smart fortwo coupe softouch/F1/Klima/Panorama,70000,...,benzin,smart,nein,20160312,0,33729,20160312,165810,20160315,31628
4,privat,Angebot,1350,test,kombi,2003,manuell,0,Focus 1 6 Benzin TÜV neu ist sehr gepflegt.mit...,150000,...,benzin,ford,nein,20160401,0,39218,20160401,143850,20160401,143850


In [68]:
#reorder the columns 
autos= autos[["brand", "model","seller","offer_type","price","abtest","vehicle_type","registration_year","gearbox","power_PS","odometer_km","registration_month","fuel_type","unrepaired_damage","ad_created","nr_of_pictures","postal_code","date_crawled_day","date_crawled_time", "last_seen_day", "last_seen_time"]]

autos.head()

,brand,model,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,...,registration_month,fuel_type,unrepaired_damage,ad_created,nr_of_pictures,postal_code,date_crawled_day,date_crawled_time,last_seen_day,last_seen_time
0,peugeot,807 160 NAVTECH ON BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,...,3,lpg,nein,20160326,0,79588,20160326,174746,20160406,64554
1,bmw,740i 4 4 Liter HAMANN UMBAU Mega Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,...,6,benzin,nein,20160404,0,71034,20160404,133856,20160406,144508
2,volkswagen,Golf 1.6 United,privat,Angebot,8990,test,limousine,2009,manuell,102,...,7,benzin,nein,20160326,0,35394,20160326,185724,20160406,201537
3,smart,smart fortwo coupe softouch/F1/Klima/Panorama,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,...,6,benzin,nein,20160312,0,33729,20160312,165810,20160315,31628
4,ford,Focus 1 6 Benzin TÜV neu ist sehr gepflegt.mit...,privat,Angebot,1350,test,kombi,2003,manuell,0,...,7,benzin,nein,20160401,0,39218,20160401,143850,20160401,143850


## Damage vs price
How much cheaper are cars with damage than their non-damaged counterparts?

In [77]:
autos["unrepaired_damage"].value_counts(normalize= True)

nein    0.871987
ja      0.128013
Name: unrepaired_damage, dtype: float64

There are 87% of cars with no damage and 13% of cars with damage. 

In [78]:
mask_1 = autos["unrepaired_damage"] == "ja"
mask_2 = autos["unrepaired_damage"] == "nein"
damage_ja = autos[mask_1]
mean_ja = damage_ja["price"].mean()

damage_nein = autos[mask_2]
mean_nein = damage_nein["price"].mean()

print("the mean price for cars without damage is", round(mean_nein,2), ", and the mean price for cars with damage is ", round(mean_ja,2),".")

the mean price for cars without damage is 4828.97 , and the mean price for cars with damage is  1962.3 .


## Mileage vs price
Split the odometer_km into groups, see if average prices follows any patterns based on the milage.

In [93]:
autos["odometer_km"].value_counts(dropna=False)

150000    28793
125000     4426
100000     1725
90000      1401
80000      1096
70000       881
60000       805
50000       661
5000        595
40000       493
30000       447
20000       398
10000       101
Name: odometer_km, dtype: int64

Suggested groups:

* < 20000
* 20001 - 50000
* 50001 - 100000
* 100000 - 125001 
* 125000 - 150000

In [105]:
#create a new column with km status

def km_map(col):
    if col <= 20000:
        return "below_20k"
    if 50000 >= col > 20001:
        return "between_20k_50k"
    if 100000 >= col > 50001:
        return "between_50k_100k"
    if 125000 >= col > 100001:
        return "between_100k_125k"
    if 150000 >= col > 125001:
        return "between_125k_150k"
    
autos["km_status"] = autos["odometer_km"].apply(km_map)
autos["km_status"].value_counts(dropna=False)

between_125k_150k    28793
between_50k_100k      5908
between_100k_125k     4426
between_20k_50k       1601
below_20k             1094
Name: km_status, dtype: int64

In [113]:
#group by price & calc mean 
grouped= autos.groupby('km_status')
grouped['price'].agg(np.mean)

km_status
below_20k            4932.827239
between_100k_125k    4856.477406
between_125k_150k    3278.142708
between_20k_50k      8252.512180
between_50k_100k     6334.855112
Name: price, dtype: float64

**Obeservations:**
* Cars with mileage between 20000 and 50000km get sold for the highest mean price. 
* The second highest price is that of cas with mileage between 50000 and 100000km.
* This is surprising sicne we might expect cars withthe lowest mileage to be sold for the highest prices. 

Unsurprisingly, the cars witht he highest mileage get the lowest mean prices. 

*However, the datasets for each group were* **not equal in size**, *which might be skewing the results.*